In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier

# Importing Data

In [2]:
data = pickle.load(open('face_race_classifier.pickle', mode='rb'))

In [3]:
X = np.array(data['data'])
y = np.array(data['label'])

In [4]:
X.shape, y.shape

((23951, 1, 128), (23951,))

In [5]:
X = X.reshape(-1,128)
X.shape

(23951, 128)

# Splitting Data Into Training and Testing Set

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((19160, 128), (4791, 128), (19160,), (4791,))

In [8]:
print(X_test)

[[ 0.05464476  0.12292917 -0.08612551 ... -0.0102466  -0.06801103
  -0.03805546]
 [ 0.04143852  0.14590509 -0.17288159 ...  0.12116858 -0.0226652
  -0.04873779]
 [ 0.0486901   0.05385214 -0.18590833 ...  0.04682554 -0.08293418
   0.01891936]
 ...
 [ 0.05954341  0.03757431 -0.0939329  ...  0.08448446 -0.01788467
   0.00273168]
 [ 0.08835659  0.10272589 -0.09891588 ... -0.06283463  0.02387561
  -0.00873818]
 [ 0.01528212  0.19012769 -0.00685082 ... -0.03440923  0.143013
   0.03368152]]


In [9]:
print(y_test)

['Asian' 'Hispanic or Latino' 'Middle Eastern' ... 'Hispanic or Latino'
 'Asian' 'Asian']


# Training Model

## Logistic Regression

In [10]:
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
def get_report(model, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)

    # F1 Score
    f1_score_train = f1_score(y_train, y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')

    print('Accuracy Train =', accuracy_train)
    print('Accuracy Test =', accuracy_test)
    print('F1 Score Train =', f1_score_train)
    print('F1 Score Test =', f1_score_test)

In [12]:
get_report(logistic_classifier, X_train, y_train, X_test, y_test)

Accuracy Train = 0.5296450939457202
Accuracy Test = 0.517011062408683
F1 Score Train = 0.5196518223448373
F1 Score Test = 0.5090158710638416


## Support Vector Machine

In [13]:
model_svc = SVC(probability=True)
model_svc.fit(X_train, y_train)

SVC(probability=True)

In [14]:
get_report(model_svc, X_train, y_train, X_test, y_test)

Accuracy Train = 0.5948851774530272
Accuracy Test = 0.5374660822375287
F1 Score Train = 0.593030821982325
F1 Score Test = 0.5371224039719548


## Random Forest

In [15]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

RandomForestClassifier()

In [16]:
get_report(model_rf, X_train, y_train, X_test, y_test)

Accuracy Train = 0.9999478079331942
Accuracy Test = 0.5113754957211438
F1 Score Train = 0.999947923659518
F1 Score Test = 0.5082373748255912


# Voting Classifier

In [17]:
model_voting = VotingClassifier(estimators = [
    ('logisitc', LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [18]:
model_voting.fit(X_train,y_train)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logisitc', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [19]:
get_report(model_voting, X_train, y_train, X_test, y_test)

Accuracy Train = 0.7080897703549061
Accuracy Test = 0.533082863702776
F1 Score Train = 0.7058467429089235
F1 Score Test = 0.5282251282050493


# Parameter Tuning

In [20]:
model_grid = GridSearchCV(model_voting,param_grid={
    'svm__C':[3,5,7,10],
    'svm__gamma':[0.1,0.3,0.5],
    'rf__n_estimators':[5,10,20],
    "rf__max_depth":[3,5,7],
    'voting':['soft','hard']
},scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [21]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time= 1.2min


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time= 1.2min


...

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time= 1.2min


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.5, voting=hard; total time= 1.2min


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logisitc',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [22]:
model_best_estimator = model_grid.best_estimator_

In [23]:
model_grid.best_score_

0.5464511712633743

# Saving Model

In [24]:
pickle.dump(model_best_estimator,open('./models/face_race.pkl',mode='wb'))